In [1]:
%load_ext autoreload
%autoreload 2
import logging
import sys
import yaml


sys.path.append('../src')
from experiments.experiment_utils import ExperimentPipeline
from tsvae.utils.logger_utils import get_console_logger
logger = get_console_logger(__name__)
from os import path as pt

In [2]:
from experiments.experiment_utils import get_output_dir, update_config
from tsvae.dataset.data_pipeline import DataPipeline
from tsvae.models.network_pipeline import NetworkPipeline
from tsvae.utils.random_utils import set_seed


base_output_dir = '..' 
new_config = {'epochs' : 100, 'wandb': True, 'comment': 'test'}
exp_config_path = '../exp_configs/expMultiBSprice/infocvae.yaml'

with open(exp_config_path) as file:
    exp_config = dict(yaml.safe_load(file))

exp_config = update_config(exp_config, new_config)

exp_config.base_output_dir = base_output_dir
exp_config.output_dir = get_output_dir(exp_config)
logger.info(f"Experiment results saved to {exp_config.output_dir}")

logger.info(f"Saving experiment config to {exp_config.output_dir}")
config_file_path = pt.join(exp_config.output_dir, "exp_config.yaml")
with open(config_file_path, "w") as outfile:
    yaml.dump(exp_config, outfile, default_flow_style=False)
# logger.info(exp_config)

# Generating data
logger.info(f"Setting ramdom seed: {exp_config.seed}")
set_seed(exp_config.seed)

Experiment results saved to ../results/BS2price_timestep_60/model_InfoCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Saving experiment config to ../results/BS2price_timestep_60/model_InfoCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Setting ramdom seed: 0


In [3]:
logger.info(f"Loading dataset: {exp_config.dataset}")
data_pipeline = DataPipeline()
train_dataset, eval_dataset = data_pipeline(exp_config)

# Loading network
logger.info("Load networks:")
network_pipeline = NetworkPipeline()
model = network_pipeline(exp_config)
# logger.info(f"{model}")

Loading dataset: BS2price
Base dataset initialized
Load networks:


In [4]:
from tsvae.trainers.base_trainer_config import BaseTrainerConfig
from tsvae.trainers.training_pipeline import TrainingPipeline


training_config = BaseTrainerConfig(
    output_dir=exp_config.output_dir,
    learning_rate=exp_config.lr,
    per_device_train_batch_size=exp_config.train_batch_size,
    per_device_eval_batch_size=exp_config.eval_batch_size,
    optimizer_cls=exp_config.optimizer,
    optimizer_params=None,
    scheduler_cls=None,
    scheduler_params=None,
    steps_saving=exp_config.steps_saving,
    steps_predict=exp_config.steps_predict,
    seed=exp_config.seed,
    num_epochs=exp_config.epochs,
    wandb_callback=exp_config.wandb,
    wandb_output_dir=exp_config.base_output_dir + "/wandb",
)

train_pipeline = TrainingPipeline(model=model, training_config=training_config, exp_config=exp_config)

# log_output_dir is not accessible through pipeline
trainer = train_pipeline(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    device_name=exp_config.device_name,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: justinhou95 (justinhou). Use `wandb login --relogin` to force relogin


Model passed sanity check !
Ready for training.



In [5]:
train_pipeline.train(log_output=True)

Created ../results/BS2price_timestep_60/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test/BetaCVAE_training_2024-10-08_20-56-58. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 100
 - per_device_train_batch_size: 256
 - per_device_eval_batch_size: 256
 - checkpoint saving every: 20
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
Scheduler: None
Device: cpu

Successfully launched training !

Training of epoch 1/100:  90%|█████████ | 9/10 [00:00<00:00, 11.87batch/s]recon: 59.87
kl: 3.07
total loss: 59.99
Eval of epoch 1/100:  10%|█         | 1/10 [00:00<00:02,  3.69batch/s]


{'train/epoch_loss': 64.38598747253418, 'eval/mmd_real_fake': tensor(0.4449), 'eval/swd_real_fake': tensor(0.3616), 'eval/mmd_real_recon': tensor(0.4445), 'eval/swd_real_recon': tensor(0.3612)}


--------------------------------------------------------------------------
Train loss: 64.386
--------------------------------------------------------------------------
Training of epoch 2/100:  90%|█████████ | 9/10 [00:00<00:00, 12.15batch/s]recon: 52.47
kl: 2.61
total loss: 52.58
Eval of epoch 2/100:  10%|█         | 1/10 [00:00<00:02,  3.52batch/s]
--------------------------------------------------------------------------
Train loss: 54.1237
--------------------------------------------------------------------------


{'train/epoch_loss': 54.123695373535156, 'eval/mmd_real_fake': tensor(0.4391), 'eval/swd_real_fake': tensor(0.3786), 'eval/mmd_real_recon': tensor(0.4305), 'eval/swd_real_recon': tensor(0.3735)}


Training of epoch 3/100: 100%|██████████| 10/10 [00:00<00:00, 12.80batch/s]recon: 46.29
kl: 5.41
total loss: 46.50
Eval of epoch 3/100:  10%|█         | 1/10 [00:00<00:02,  3.75batch/s]
--------------------------------------------------------------------------
Train loss: 47.5157
--------------------------------------------------------------------------


...

Training of epoch 99/100: 100%|██████████| 10/10 [00:00<00:00, 12.98batch/s]recon: 4.17
kl: 86.15
total loss: 7.61
Eval of epoch 99/100:  10%|█         | 1/10 [00:00<00:02,  3.84batch/s]
--------------------------------------------------------------------------
Train loss: 7.5464
--------------------------------------------------------------------------


{'train/epoch_loss': 7.546445322036743, 'eval/mmd_real_fake': tensor(0.0328), 'eval/swd_real_fake': tensor(0.1000), 'eval/mmd_real_recon': tensor(0.0002), 'eval/swd_real_recon': tensor(0.0227)}


Training of epoch 100/100:  90%|█████████ | 9/10 [00:00<00:00, 12.45batch/s]recon: 4.17
kl: 84.76
total loss: 7.56
Eval of epoch 100/100:  10%|█         | 1/10 [00:00<00:07,  1.27batch/s]
Saved checkpoint at epoch 100

--------------------------------------------------------------------------
Train loss: 7.5583
--------------------------------------------------------------------------
Training ended!
Saved final model in ../results/BS2price_timestep_60/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test/BetaCVAE_training_2024-10-08_20-56-58/final_model


{'train/epoch_loss': 7.558252620697021, 'eval/mmd_real_fake': tensor(0.0333), 'eval/swd_real_fake': tensor(0.0916), 'eval/mmd_real_recon': tensor(0.0003), 'eval/swd_real_recon': tensor(0.0236)}


eval/mmd_real_fake,▇▇▇▇██▄▃▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/mmd_real_recon,█▇▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/swd_real_fake,██▇▇▇▇▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▂▁▁▂▂▂▂
eval/swd_real_recon,██▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
eval/mmd_real_fake,0.03332
eval/mmd_real_recon,0.00029
eval/swd_real_fake,0.09162
eval/swd_real_recon,0.02361
train/epoch_loss,7.55825
